In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os

In [7]:
def count_missing(fullfilename):
    df = pd.read_csv(fullfilename)
    print(df.isna().sum())

In [ ]:
def create_map(fullfilename):
    df = pd.read_csv(fullfilename)
    

In [10]:
filepath = "ncbi\\aka_scraped"

for filename in os.listdir(filepath):
    ffpath = os.path.join(filepath, filename)
    df_tmp = pd.read_csv(ffpath)
    print()
    print(filename)
    count_missing(ffpath)
    print()


1Colorectal.csv
LncRNA name                                                          0
Cancer name                                                          2
Methods                                                              2
Expression pattern                                                   4
Pubmed ID                                                            2
Remarks                                                             93
No. of Transcript variants                                           2
NCBI Reference Number                                                2
Total No. of PQS                                                     2
No. of 2G PQS                                                        2
No. of 3G PQS                                                        2
No. of 4G PQS                                                        2
RefSeq status                                                        2
Total No. of PQS.1                                          

In [11]:
colopath = os.path.join(filepath, '1Colorectal.csv')
print(colopath)

ncbi\aka_scraped\1Colorectal.csv


In [14]:
df = pd.read_csv(colopath)
print(df.columns)

Index(['LncRNA name', 'Cancer name', 'Methods', 'Expression pattern',
       'Pubmed ID', 'Remarks', 'No. of Transcript variants',
       'NCBI Reference Number', 'Total No. of PQS', 'No. of 2G PQS',
       'No. of 3G PQS', 'No. of 4G PQS', 'RefSeq status', 'Total No. of PQS.1',
       'G4Hunter Scores of PQS',
       'Similarity between PQS from QGRS mapper vs PQS from G4 hunter',
       'Total No. of PQS.2', 'G4Hunter Scores of PQS.1',
       'Similarity between PQS from QGRS mapper vs PQS from G4 hunter.1',
       'aka'],
      dtype='object')


In [15]:
df = pd.read_csv(colopath)

cols = ['LncRNA name', 'Cancer name', 'Methods', 'Expression pattern',
       'Pubmed ID', 'aka']

In [19]:
df[cols][:-2]

,LncRNA name,Cancer name,Methods,Expression pattern,Pubmed ID,aka
0,ABHD11-AS1,colorectal cancer,"qPCR, RIP, Luciferase reporter assay, Western ...",up-regulated,30429229.0,WBSCR26; LINC00035; NCRNA00035
1,ABHD11-AS1,colorectal cancer,"qPCR, Western blot, Luciferase reporter assay,...",up-regulated,30537177.0,WBSCR26; LINC00035; NCRNA00035
2,ADAMTS9-AS2,colorectal cancer,qPCR etc.,down-regulated,27596298.0,ADAMTS9 antisense RNA 2;ADAMTS9 antisense RNA ...
3,ADPGK-AS1,colorectal cancer,"qPCR, Luciferase reporter assay etc",up-regulated,32196589.0,ADPGK antisense RNA 1;ADPGK antisense RNA 1 (n...
4,AFAP1-AS1,colorectal cancer,"qPCR, Western blot, in vitro knockdown, RIP",up-regulated,29888110.0,AFAP1AS; AFAP1-AS
...,...,...,...,...,...,...
661,ZFAS1,colon cancer,"qPCR, Western blot, Flow cytometry assay, Cell...",up-regulated,27862275.0,HSUP1; HSUP2; C20orf199; ZNFX1-AS1; NCRNA00275
662,ZFAS1,colorectal cancer,"qPCR, Luciferase reporter assay",up-regulated,29179614.0,HSUP1; HSUP2; C20orf199; ZNFX1-AS1; NCRNA00275
663,ZFAS1,colorectal cancer,qPCR,up-regulated,28406230.0,HSUP1; HSUP2; C20orf199; ZNFX1-AS1; NCRNA00275
664,ZFAS1,colorectal cancer,"qPCR, RNAi, Cell migration and invasion assay ...",up-regulated,27461828.0,HSUP1; HSUP2; C20orf199; ZNFX1-AS1; NCRNA00275


In [49]:
def get_dups(fullfilepath):    
    df = pd.read_csv(fullfilepath)

    cols = ['LncRNA name', 'Cancer name', 'Methods', 'Expression pattern',
        'Pubmed ID', 'aka']

    present_map = {}
    dup_indices = []
    for index, row in df[cols][:-2].iterrows():
        curentry = tuple(row[['LncRNA name', 'Cancer name', 'Methods', 'Expression pattern',
        'Pubmed ID',]])
        present_map[curentry] = (False, [])
        # akas = str(list(row[['aka']])[0]).strip().split(';')
        
        # for aka in akas:
        #     tmp_list = list(curentry)
        #     tmp_list[0] = aka

    for index, row in df[cols][:-2].iterrows():
        curentry = tuple(row[['LncRNA name', 'Cancer name', 'Methods', 'Expression pattern',
        'Pubmed ID',]])
        
        if present_map[curentry][0] == True:
            # dup_indices.append((index, present_map[curentry][1]))
            present_map[curentry][1].append(index)
        else:
            akas = str(list(row[['aka']])[0]).strip().split(';')

            for aka in akas:
                tmplist = list(curentry)
                tmplist[0] = aka
                akaentry = tuple(tmplist)

                if akaentry in present_map.keys() and present_map[akaentry][0] == True:
                    # dup_indices.append((index, present_map[curentry][1]))
                    present_map[akaentry][1].append(index)

            present_map[curentry] = (True, [index])

    return {fullfilepath: present_map}

In [68]:
delete_rows = {}

for filename in os.listdir(filepath):
    fullfilename = os.path.join(filepath, filename)
    dups_data = get_dups(fullfilename)

    removerows = []

    for i in dups_data.values():
        for j in i:
            if len(i[j][1]) > 1:
                for rownum in i[j][1][1:]:
                    removerows.append(rownum)

    delete_rows[filename] = removerows

import json
with open('final_dups_to_remove.json', 'w') as fp:
    json.dump(delete_rows, fp)

# remove row number+2 wali row in excel from the sheet